In [ ]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')

from model.dropout_uncertainty_enc_dec_LSTM_new_dec.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [ ]:
#load model
file_path_model = '../training_new_dec/BPIC17/BPIC_2017_full_grad_norm_new_1.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../encoded_data/BPIC_2017_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

In [ ]:
import src.evaluation.probabilistic_evaluation
importlib.reload(src.evaluation.probabilistic_evaluation)
from src.evaluation.probabilistic_evaluation import ProbabilisticEvaluation

#device = torch.device("cuda")  # Use the first available GPU
#model = model.to(device)

new_eval = ProbabilisticEvaluation(model, bpic_17_test_dataset,
                                   num_processes=16,
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   )


In [ ]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'cont_results_part_{chunk_number:05d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")


with open('BPIC2017_processed_prefixes.pkl', 'rb') as f:
    processed_prefixes = pickle.load(f)

output_dir = '../../../../data/BPIC17/gn_continue'
save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) \
in enumerate(new_eval.evaluate_continue_multi_processing(processed_prefixes = processed_prefixes,
                                                         random_order=False)):
    print('Process', case_name, prefix_len)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)